### Goals of this notebook

first, we need to define clearly how the user inputs will look like,
 
then, we will be able to code a function that takes the user inputs and returns a pandas df with an extra ["country_user_score"] column

but before coding the actual python scripts, it's easier to just simulate that flow in a notebook

user inputs structure:
1. climate
    climate preference -> selects category between [cold, warm, hot]
    climate - how important is this for you? -> slider
2. safety
    safety -> [average, safe, very safe]
    safety - how important is this for you? -> slider
3. healthcare
    healthcare quality -> [average, good, great]
    healthcare quality - how important is this for you? -> slider
4. cost of living
    Monthly cost preference -> [cheap, medium, expensive]
    cost of living - how important is this for you? -> slider
    optional: filter by a monthly cost budget -> give a max monthly budget in dollars
5. Internet speed
    Internet speed preference -> [medium, fast, very fast]
    internet speed - how important is this for you? -> slider


------

*structure of the API endpoint*
1. user sends get request to the API with user inputs as parameters
2. API function calls the "process_user_input" class / module
3. "process_user_input" module takes user inputs and processes to make them "understandable" to the dataset data and ready to insert in the weighted sum
4. API function calls the "weighted_sum" class / module
5. "weighted_sum" class / module takes the numerical inputs and calculates the the ["country_user_score"] column for each country - returns the pandas df
6. lastly, API function returns the head(5) of the df sorted by country_user_score

## Pipeline